In [164]:
import pytesseract
from pdf2image import convert_from_path
import re
import numpy
import pandas
from functools import reduce

In [190]:
mass_pattern = re.compile(r'\d{1,3}(?:[, \d]*\d)?(?:\.\d+)?', re.UNICODE)
species_pattern = re.compile(r'([A-Z][a-z]+(?:\s[a-z]+)?)', re.UNICODE)

In [ ]:
pages = convert_from_path('/home/andrewstier/Downloads/The Company of Biologists.pdf', first_page=1, last_page=35)
data = []
# fix the species name going to a second line
for page in pages:
    extracted_text = pytesseract.image_to_string(page)
    try:
        species_candidates = [x.group(0) for x in list(filter(None,[re.match(species_pattern,line) for line in  extracted_text.split('\n')]))]        
        keep_species = numpy.array([len(species_candidates[i].split(' '))==2 or ((i>1) and ((len(species_candidates[i-1].split(' '))==2) and (len(species_candidates[i+1].split(' '))==2)))  for i in range(len(species_candidates)-1)])
        keep_end = numpy.argwhere(numpy.diff(numpy.argwhere(keep_species).flatten())>1)
        keep_end = min(len(species_candidates),keep_end.flatten()[0]+2) if len(keep_end)>0 else len(species_candidates)
        species = species_candidates[numpy.argwhere(keep_species).flatten()[0]:keep_end]

        mass_candidates = [x.group(0) for x in list(filter(None,[re.match(mass_pattern,line) for line in  extracted_text.split('\n')]))]
        keep_mass = numpy.array([len(x.split('.'))>1 for x in mass_candidates])
        mass = mass_candidates[numpy.argwhere(keep_mass).flatten()[0]:]
        mass = mass[:len(species)]

        metabolism_candidates = [x.group(0) for x in list(filter(None,[re.match(mass_pattern,line) for line in  extracted_text.split('\n')]))]
        metabolism = list(filter(lambda x: len(x.split(' '))==1 and len(x.split('.'))==2 and not x.__contains__(','),metabolism_candidates))
        metabolism = metabolism[-len(mass):]
        data.append([species,mass,metabolism])
    except:
        pass

data[-2] = [data[-2][0][:-1],data[-2][1][:-1],data[-2][2]]
manual_data = [x.split('\n') for x in ["Dobsonia minor 87.00 0.62\nEonycteris spelaea 52.00 0.27\nEptesicus fuscus 16.90 0.11",
"Priodontes maximus 45 190.00 17.05\nTamandua mexicana 4210.00 4.79\nTamandua tetradactyla 3500.00 4.99\nTolypeutes matacus 1160.00 1.18",
"Lepus californicus 2 300.00 7.38\nOchotona daurica 127.70 1.40\nOchotona princeps 109.00 0.94",
"Macropus eugenii 4796.00 7.78\nMacropus robustus 30000.00 34.46\nMacrotis lagotis 1477.00 2.79\nMarmosa microtarsus 13.00 0.13",
"Manis crassicaudata 15 910.00 6.99\nManis javanica 4220.00 6.23\nManis tricupis 2730.00 4.63",
"Citellus undulatus 680.00 3.31\nClethrionomys gapperi 23.00 0.26\nClethrionomys glareolus 20.50 0.27\nClethrionomys rufocanus 27.50 0.31",
"eliophobius kapeti 89.00 0.44\nHeliosciurus rufobrachium 271.00 0.90\nHeterocephalus glaber 39.00 0.14",
"Spermophilus undulatus 1000.00 4.50\nStochomys longicaudatus 82.33 0.55\nTachyoryctes splendens 234.00 0.92\nTamias amoenus 57.00 0.54\nTamias merriami 75.00 0.44\nTamias striatus 87.40 0.51\nTamiasciurus hudsonicus 224.00 1.41\nTatera indica 86.80 0.43\nThomomys bottae 143.00 0.68\nThomomys talpoides 82.60 0.55\nThomomys umbrinus 95.30 0.46\nTrichomys apereoides 323.00 1.16\nZapus hudsonicus 30.00 0.25\nPtilocercus lowii 57.50 0.24\nTrichechus inunguis 170.000.00 46.91"
]]
manual_data = [x for xs in manual_data for x in xs]
species = [x[0]+' '+x[1] for x in [x.split(' ')[:2] for x in manual_data]]
mass = [(reduce(lambda a,b: a+b,x[0].split(' ')[2:]) +'.'+x[1].split(' ')[0]).replace(' ','') for x in  [x.split('.') for x in manual_data]]
b = [x.split(' ')[-1] for x in manual_data]
data.append([species,mass,b])

In [ ]:
df = pandas.concat([pandas.DataFrame(numpy.vstack(x).T,columns=['species','M(g)','B(W)']) for x in data])
df['M(g)'] = df['M(g)'].map(lambda x: x.replace(' ','').replace(',','')).astype(float)
df['B(W)'] = df['B(W)'].map(lambda x: x.replace(' ','').replace(',','')).astype(float)

In [187]:
df.to_csv('data/metabolism_mass_mammals.csv')